In [58]:
import matplotlib.pyplot as plt  
import pandas as pd  
import numpy as np 
import seaborn as sns
import os, sys
os.chdir("/content/drive/My Drive/phd/")
import tensorflow as tf
from sklearn.metrics import homogeneity_completeness_v_measure
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from scipy.stats import pearsonr
from sklearn.decomposition import LatentDirichletAllocation
sys.path.append('/content/drive/My Drive/phd/master_thesis/hsbm/')
sys.path.append('/content/drive/My Drive/phd/master_thesis')
from hsbmpy import get_file, define_labels, get_cluster_given_l, get_max_available_L
!pip install gseapy
from geneontology import topic_analysis
from lda import lda

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import lda, importlib
importlib.reload(lda)
from lda import lda

In [0]:
level = 2
#setup = 'oversigma_10tissue'
#label = 'disease_type'
label='topsbm'
#label = 'uniq'
#labels = ['primary_site', 'disease_type']
#labels = ['primary_site', 'secondary_site']
directory=r"/content/drive/My Drive/phd/datasets/gtex/10"
#L=get_max_available_L(directory)
os.chdir(directory)

In [91]:
df = pd.read_csv("mainTable_hv.csv", index_col=[0], header=[0]).dropna().astype(int)
totalobjcets = len(df.columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2999 entries, ENSG00000268903.1 to ENSG00000210196.2
Columns: 1000 entries, GTEX-1GZ2Q-0011-R6a-SM-9OSWM to GTEX-RM2N-0526-SM-2TF4N
dtypes: int64(1000)
memory usage: 22.9+ MB


In [0]:
df.to_csv("mainTable.csv", index=True, header=True)

In [0]:
df = df.applymap(lambda tpm: np.log2(tpm+1))
df.to_csv("mainTable.csv", index=True, header=True)

In [93]:
df_files = pd.read_csv("files.dat", index_col=[0])
df_files.head()

,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,SMNABTCH,SMNABTCHT,SMNABTCHD,SMGEBTCH,SMGEBTCHD,SMGEBTCHT,SMAFRZE,SMGTC,SME2MPRT,SMCHMPRS,SMNTRART,SMNUMGPS,SMMAPRT,SMEXNCRT,SM550NRM,SMGNSDTC,SMUNMPRT,SM350NRM,SMRDLGTH,SMMNCPB,SME1MMRT,SMSFLGTH,SMESTLBS,SMMPPD,SMNTERRT,SMRRNANM,SMRDTTL,SMVQCFL,SMMNCV,SMTRSCPT,SMMPPDPR,SMCGLGTH,SMGAPPCT,SMUNPDRD,SMNTRNRT,SMMPUNRT,SMEXPEFF,SMMPPDUN,SME2MMRT,SME2ANTI,SMALTALG,SME2SNSE,SMMFLGTH,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
SAMPID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GTEX-111CU-1426-SM-5GZYP,1.0,B1,"6 pieces, predominantly mucosa and submucosa w...",7.8,Colon,Colon - Transverse,0001157,100.0,609.0,BP-43693,RNA Extraction from Paxgene-derived Lysate Pla...,09/17/2013,LCSET-4804,03/05/2014,TruSeq.v1,RNASEQ,NaN,0.989352,229793.0,0.975903,NaN,0.992480,0.817932,NaN,22970.0,1.0,NaN,76.0,NaN,0.002257,229.0,0.0,80139500.0,0.023389,634774.0,80746700.0,10086300.0,NaN,22993.0,39840000.0,NaN,NaN,0.0,0.157971,0.992480,0.811782,80139500.0,0.003262,17876700.0,11387200.0,17794400.0,235.0,17849000.0,13405900.0,0.002758,18057000.0,50.2897,0.007861,0.995609,NaN,0.0,49.8847
GTEX-111FC-0226-SM-5N9B8,2.0,B1,"2 pieces, larger piece is 30% fibrovascular ti...",7.3,Adipose Tissue,Adipose - Subcutaneous,0002190,1040.0,869.0,BP-43989,RNA Extraction from Paxgene-derived Lysate Pla...,09/26/2013,LCSET-4904,03/22/2014,TruSeq.v1,RNASEQ,NaN,0.992062,167541.0,0.974332,NaN,0.992977,0.812219,NaN,23977.0,1.0,NaN,76.0,NaN,0.002809,282.0,0.0,94641900.0,0.025325,357934.0,95311300.0,4853830.0,NaN,24018.0,47128600.0,NaN,NaN,0.0,0.162113,0.992977,0.806515,94641900.0,0.003400,20778300.0,8509160.0,20875200.0,256.0,20831300.0,17554200.0,0.003104,20911100.0,50.0955,0.003755,0.993892,NaN,0.0,50.1164
GTEX-111YS-0006-SM-5NQBE,NaN,B1,NaN,8.2,Blood,Whole Blood,0013756,-121.0,NaN,BP-38780,RNA isolation_PAXgene Blood RNA (Manual),05/08/2013,LCSET-4907,03/21/2014,TruSeq.v1,RNASEQ,NaN,0.981573,565100.0,0.965938,NaN,0.985108,0.810071,NaN,18229.0,1.0,NaN,76.0,NaN,0.002712,128.0,0.0,79518700.0,0.032955,224915.0,80720700.0,11154300.0,NaN,18237.0,39500100.0,NaN,NaN,0.0,0.155867,0.985108,0.798007,79518700.0,0.005861,17750400.0,69342100.0,18023700.0,148.0,18081500.0,17940700.0,0.004281,17976300.0,49.8542,0.002786,0.988644,NaN,0.0,50.3820
GTEX-111YS-0326-SM-5GZZ3,1.0,B1,"2 pieces, 1 with 50% external fat; hypertrophi...",8.5,Heart,Heart - Atrial Appendage,0006631,120.0,769.0,BP-43693,RNA Extraction from Paxgene-derived Lysate Pla...,09/17/2013,LCSET-4804,03/05/2014,TruSeq.v1,RNASEQ,NaN,0.987427,263296.0,0.982231,NaN,0.990372,0.836055,NaN,20649.0,1.0,NaN,76.0,NaN,0.002170,161.0,0.0,72421400.0,0.017499,1662590.0,73125500.0,8414410.0,NaN,20663.0,35962300.0,NaN,NaN,0.0,0.146176,0.990372,0.828006,72421400.0,0.003791,15416200.0,19062400.0,15467900.0,190.0,15310400.0,11400800.0,0.002978,15763800.0,50.7295,0.022736,0.993318,NaN,0.0,50.0837
GTEX-111YS-2426-SM-5GZZQ,0.0,B1,3 small irregular pieces,6.6,Adipose Tissue,Adipose - Subcutaneous,0002190,296.0,593.0,BP-43693,RNA Extraction from Paxgene-derived Lysate Pla...,09/17/2013,LCSET-4805,02/27/2014,TruSeq.v1,RNASEQ,NaN,0.990443,250797.0,0.974452,NaN,0.992890,0.799713,NaN,21965.0,1.0,NaN,76.0,NaN,0.002132,159.0,0.0,68259500.0,0.025234,457318.0,68748200.0,7724790.0,NaN,21987.0,33965000.0,NaN,NaN,0.0,0.174739,0.992890,0.794027,68259500.0,0.003396,15047300.0,9352700.0,15104000.0,165.0,15144200.0,10941600.0,0.002763,15171800.0,50.0454,0.006652,0.995338,NaN,0.0,50.0941


In [0]:
true_out = []
for sample in df.columns.values:
    try:
        true_out.append(get_file(sample, df_files)['SMTS'])
    except:
        print(*sys.exc_info())
        true_out.append('unknown')

In [0]:
scores = {}

In [85]:
try:
    with open("clustersizes.txt",'r') as f:
        xl = np.array(f.read().split(sep='\n'))[:-1].astype(int)
except:
        xl=np.linspace(2,50,5, dtype=int)
xl

array([987, 135,  20,   6,   1])

In [0]:
xl = np.sort(np.concatenate([xl,np.arange(4,100,25)]))

In [86]:
xl

array([987, 135,  20,   6,   1])

## hierarchical

In [0]:
fig=plt.figure()
dend = shc.dendrogram(shc.linkage(df.T.values, method='average'), leaf_rotation=90., leaf_font_size=8.,)
plt.xlabel("samples", fontsize=16)
plt.show()
fig.savefig("hierarchical_dendogram_log.pdf")

In [98]:
#hierarchical
scores['hierarchical']={
    'h':[],
    'c':[],
    'V':[]
}
def pearson_affinity(M, parallel=True):
    return 1 - np.array([[pearsonr(a,b)[0] for a in M] for b in M])

print("hierarchical")
os.system('mkdir -p hierarchical')
hierarchical_model = AgglomerativeClustering(n_clusters=1, affinity='euclidean', linkage='complete')  
for l,x in enumerate(xl):
    print("testing with %d clusters"%x)
    hierarchical_model.n_clusters=x
    data = df.T.values
    out = hierarchical_model.fit_predict(data)
        
    #save clusters
    print("saving clusters")
    df_clusters = pd.DataFrame(index=np.arange(len(df.columns)))
    for c in np.arange(out.max()+1)[::-1]:
        c_objects = df.columns[np.argwhere(out==c)].values.T[0]
        df_clusters.insert(0,"Cluster %d"%(c+1),np.concatenate((c_objects,[np.nan for _ in np.arange(len(df.columns)-len(c_objects))])))
    df_clusters.dropna(axis=0,how='all', inplace=True)
    df_clusters.to_csv("hierarchical/hierarchical_level_%d_clusters.csv"%(l), index=False, header=True)

hierarchical
testing with 1 clusters
saving clusters
testing with 6 clusters
saving clusters
testing with 20 clusters
saving clusters
testing with 135 clusters
saving clusters
testing with 987 clusters
saving clusters


## LDA

In [0]:
with open("topicsizes.txt",'r') as f:
    tl = np.array(f.read().split(sep='\n'))[:-1].astype(int)

In [88]:
xl=xl[::-1]
xl

array([  1,   6,  20, 135, 987])

In [96]:
model=lda(n_jobs=-1, verbose=2)
model.full_analysis(directory, xl,tl=None, label='SMTS')

model created
<class 'pandas.core.frame.DataFrame'>
Index: 2999 entries, ENSG00000268903.1 to ENSG00000210196.2
Columns: 1000 entries, GTEX-1GZ2Q-0011-R6a-SM-9OSWM to GTEX-RM2N-0526-SM-2TF4N
dtypes: int64(1000)
memory usage: 22.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, GTEX-111CU-1426-SM-5GZYP to GTEX-ZZPU-0926-SM-5GZYT
Data columns (total 62 columns):
SMATSSCR     786 non-null float64
SMCENTER     1000 non-null object
SMPTHNTS     786 non-null object
SMRIN        1000 non-null float64
SMTS         1000 non-null object
SMTSD        1000 non-null object
SMUBRID      1000 non-null object
SMTSISCH     994 non-null float64
SMTSPAX      784 non-null float64
SMNABTCH     1000 non-null object
SMNABTCHT    1000 non-null object
SMNABTCHD    1000 non-null object
SMGEBTCH     1000 non-null object
SMGEBTCHD    1000 non-null object
SMGEBTCHT    1000 non-null object
SMAFRZE      1000 non-null object
SMGTC        0 non-null float64
SME2MPRT     1000 non-null float64
SMCHMP

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Don

iteration: 1 of max_iter: 5


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Don

iteration: 2 of max_iter: 5


...

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.8s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.8s remaining:    0.0s
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.0s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Don

iteration: 5 of max_iter: 5


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    8.7s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    8.7s remaining:    0.0s
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    8.8s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    8.8s remaining:    0.0s


saving word-distr
saving topic-distr
<class 'AttributeError'> 'Tensor' object has no attribute 'numpy' <traceback object at 0x7fb09b9ccdc8>
saving clusters
saving metrics


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    9.0s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    9.0s remaining:    0.0s


In [0]:
fig, ax=plt.subplots(figsize=(20,10))
for topic in df_D.columns:
    df_D[topic].hist(ax=ax, histtype='step', density=True, lw=2, label=topic)
ax.set_yscale('log')
ax.set_xlabel('$D^g[k]$', fontsize=20)
plt.tick_params(labelsize=20)
plt.tick_params(axis='x', rotation=60)
ax.legend(ncol=5)
plt.show()
fig.savefig("%s/distinctivness.pdf"%directory)

In [0]:
topic_analysis(directory,1, 'lda')

In [0]:
Sigmas

In [0]:
homogeneity_completeness_v_measure(true_out, out)

## LDA Mallet

In [0]:
from gensim.models.wrappers import LdaMallet
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import get_tmpfile
from gensim.corpora import MmCorpus

In [0]:
corpus=list(map(list,map(lambda x: list(zip(range(len(df.index)), df[x])),df.columns)))
corpus

In [0]:
dictionary=Dictionary([df.index])
[a for a in dictionary.items()]

In [0]:
out_file=get_tmpfile("/home/fvalle/phd/datasets/tcga/oversampling_10tissue/corpus.mm")
MmCorpus.serialize(out_file, corpus, dictionary)

In [0]:
model = LdaMallet("/home/fvalle/phd/Mallet/bin/mallet", workers=5, corpus=corpus, num_topics=15, id2word=dictionary)

In [0]:
df_topics=pd.DataFrame(data=model.get_topics().T, index=[a[1] for a in dictionary.items()], columns=["Topic %d"%(t+1) for t in range(5)])
df_topics

In [0]:
df_topic_distr=pd.read_csv(model.fdoctopics(), sep='\t', header=None, index_col=0).drop(1,1)
df_topic_distr.columns=["Topic %d"%(t+1) for t in range(5)]
df_topic_distr.index=df.columns
df_topic_distr

In [0]:
df_topic_distr.apply(lambda x: x.idxmax().split(" ")[1], axis=1)

## hierachical on Altmann's output

In [0]:
hiermodel = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward')

In [0]:
with open('clustersizes.txt') as f:
    xl=np.array(f.read().split('\n')[:-1]).astype(int)

In [0]:
os.system("mkdir -p hierhsbm")

In [0]:
true_out = []
for sample in pd.read_csv("%s/%s_level_%d_topic-dist.csv"%('topsbm','topsbm',0), index_col=1).drop('i_doc', axis=1).index.values:
    try:
        true_out.append(get_file(sample, df_files)['primary_site'])
    except:
        print(sys.exc_info()[0])
        true_out.append('')

In [0]:
scores['hierhsbm']={
    'h':[],
    'c':[],
    'V':[]
}
for l,n_clusters in enumerate(xl):
    print("Fitting level %d with %d clusters"%(l, n_clusters))
    df_topics = pd.read_csv("%s/%s_level_%d_topic-dist.csv"%('topsbm','topsbm',l), index_col=1).drop('i_doc', axis=1)
    df_clusters = pd.DataFrame(columns=["Cluster %d"%c for c in np.arange(n_clusters)+1])
    hiermodel.n_clusters=n_clusters
    out = hiermodel.fit_predict(df_topics.values)  
    for c in np.arange(out.max()+1)[::-1]:
        c_objects = df_topics.index[np.argwhere(out==c)].values.T[0]
        df_clusters["Cluster %d"%(c+1)]=np.concatenate((c_objects,[np.nan for _ in np.arange(len(df_topics.index)-len(c_objects))]))
    df_clusters.dropna(axis=0,how='all', inplace=True)
    df_clusters.to_csv("hierhsbm/hierhsbm_level_%d_clusters.csv"%(l), index=False, header=True)
    #metrics
    print("saving metrics")
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['hierhsbm']['h'].append(score[0])
    scores['hierhsbm']['c'].append(score[1])
    scores['hierhsbm']['V'].append(score[2])
    
pd.DataFrame(data=scores['hierhsbm']).to_csv("%s/hierhsbm.scores"%directory, header=True, index=False)

## Topic mapping

In [0]:
df_clusters = pd.read_csv("tm/tm_level_0_clusters.csv")

In [0]:
out=[]
true_out=[]
for c in df_clusters.columns:
    for sample in df_clusters[c].dropna():
        out.append(c)
        true_out.append(get_file(sample, df_files)['primary_site'])

In [0]:
score = (*homogeneity_completeness_v_measure(true_out, out), len(df_clusters.columns))
    
pd.DataFrame(data=score, index=['h','c','v', 'n']).transpose().to_csv("%s/tm.scores"%directory, header=True, index=False)

# t-SNE

## WGCNA

In [0]:
for l in range(4):
    df_wgcna = pd.read_csv("wgcna/wgcna_level_%d_labels.csv"%l, index_col=0)
    totalobjcets = len(df_wgcna.index)
    out = df_wgcna['x'].values
    print("saving clusters")
    df_clusters = pd.DataFrame(index=np.arange(totalobjcets))
    for c in np.arange(out.max()+1)[::-1]:
        c_objects = df_wgcna.index[np.argwhere(out==(c+1))].values.T[0]
        df_clusters.insert(0,"Cluster %d"%(c+1),np.concatenate((c_objects,[np.nan for _ in np.arange(totalobjcets-len(c_objects))])))
    df_clusters.dropna(axis=0,how='all', inplace=True)
    df_clusters.to_csv("wgcna/wgcna_level_%d_clusters.csv"%l, index=False, header=True)
df_wgcna_td = pd.read_csv("wgcna/wgcna_level_0_topic-dist.csv")
df_wgcna_td.columns.values[0]='doc'
df_wgcna_td.index.name='i_doc'
df_wgcna_td.to_csv("wgcna/wgcna_level_0_topic-dist.csv", index=True)

In [0]:
df_wgcna_td = pd.read_csv("wgcna/wgcna_level_0_topic-dist.csv").drop(['doc'], axis=1)
df_wgcna_td.columns.values[1]='doc'
df_wgcna_td
df_wgcna_td.to_csv("wgcna/wgcna_level_0_topic-dist.csv", index=False)

## hSBM

In [0]:
for l in range(3):
    pd.read_csv("topsbm/topsbm_level_%d_topics.csv"%l).applymap(lambda x: str(x)[:15]).to_csv("topsbm/topsbm_level_%d_topics.csv"%l, index=False)